In [1]:
import numpy as np
import re

In [17]:
corpus = """Simple example with Cats and Mouse
Another simple example with dogs and cats
Further simple example with mouse and cheese""".split("\n")

In [18]:
corpus

['Simple example with Cats and Mouse',
 'Another simple example with dogs and cats',
 'Further simple example with mouse and cheese']

In [19]:
# clearing and tokenizing
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

# Calculating bag of words
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

word_dict_A = dict.fromkeys(word_set, 0)
print(word_dict_A)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1
print(word_dict_A)
    
for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

{'example', 'mouse', 'cats', 'and', 'further', 'another', 'dogs', 'simple', 'with', 'cheese'}
{'example': 0, 'mouse': 0, 'cats': 0, 'and': 0, 'further': 0, 'another': 0, 'dogs': 0, 'simple': 0, 'with': 0, 'cheese': 0}
{'example': 1, 'mouse': 1, 'cats': 1, 'and': 1, 'further': 0, 'another': 0, 'dogs': 0, 'simple': 1, 'with': 1, 'cheese': 0}


In [20]:
word_dict_A

{'example': 1,
 'mouse': 1,
 'cats': 1,
 'and': 1,
 'further': 0,
 'another': 0,
 'dogs': 0,
 'simple': 1,
 'with': 1,
 'cheese': 0}

In [21]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf
  
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

In [22]:
tf_A

{'example': 0.16666666666666666,
 'mouse': 0.16666666666666666,
 'cats': 0.16666666666666666,
 'and': 0.16666666666666666,
 'further': 0.0,
 'another': 0.0,
 'dogs': 0.0,
 'simple': 0.16666666666666666,
 'with': 0.16666666666666666,
 'cheese': 0.0}

In [23]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = np.log(n / float(v))
    return idf
    
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])
idf

{'example': 0.0,
 'mouse': 0.4054651081081644,
 'cats': 0.4054651081081644,
 'and': 0.0,
 'further': 1.0986122886681098,
 'another': 1.0986122886681098,
 'dogs': 1.0986122886681098,
 'simple': 0.0,
 'with': 0.0,
 'cheese': 1.0986122886681098}

In [24]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf
    
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [25]:
tf_idf_A

{'example': 0.0,
 'mouse': 0.06757751801802739,
 'cats': 0.06757751801802739,
 'and': 0.0,
 'further': 0.0,
 'another': 0.0,
 'dogs': 0.0,
 'simple': 0.0,
 'with': 0.0,
 'cheese': 0.0}

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


all_text  =  """
Google and Facebook are strangling the free press to death. Democracy is the loserGoogle an 
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
""".split("\n")[1:-1]

# Preprocessing and tokenizing
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]", " ", line)
    return line

In [33]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)
tfidf

<13x94 sparse matrix of type '<class 'numpy.float64'>'
	with 141 stored elements in Compressed Sparse Row format>

In [36]:
kmeans = KMeans(n_clusters=4).fit(tfidf)

In [38]:
# lines_for_predicting the cluster for given texts
lines_for_predicting = ["tf and idf is awesome!", "Selling Your Soul to Google"]
kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

array([3, 1])